In [9]:
%load_ext autoreload
%autoreload 2

In [12]:
from train import LitTransformer
from pathlib import Path
from data import TransformerDataset
import torch
import yaml
from data import get_tgt_str, id2token
import pandas as pd

from torch import nn
import pandas as pd
from data import TransformerDataset

model_path = "/home/takeru/AlphaSymbol/models/d3-a5-c3-r5/2024-1218-0225-50/lightning_logs/version_0/checkpoints/epoch=633-step=22190.ckpt"

model = LitTransformer.load_from_checkpoint(model_path)
device = model.device
model.to(device)
model.eval()

print()

# Generate whole string

In [20]:

src_vocab = model.src_vocab
tgt_vocab = model.tgt_vocab
src_max_len = model.src_max_len
tgt_max_len = model.tgt_max_len

inputs = [(0,), (1,), (2,), (3,), (4,), (5,), (6,), (7,), (8,), (9,)]
#inputs = [(0,), (0,), (0,), (0,), (0,), (0,), (0,), (0,), (0,), (0,)]
#inputs = [(0,), (0,)]

#outputs = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
#outputs = [2, 3, 4, 5, 6, 7, 8, 9, 10, 11]
#outputs = [3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
#outputs = [4, 5, 6, 7, 8, 9, 10, 11, 12, 13]
#outputs = [6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16]
#outputs = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
#outputs = [0, 2, 4, 6, 8, 10, 12, 14, 16, 18]
outputs = [0, 0, 2, 4, 6, 8, 10, 12, 14, 16]

#outputs = [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
#outputs = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
#outputs = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
#outputs = [0, 0]
#outputs = [1, 1]
#outputs = [2, 2]
#outputs = [3, 3]


src_str = get_tgt_str(inputs, outputs)
src_ends = torch.tensor([src_vocab["<sos>"]] + [src_vocab[i] for i in src_str] + [src_vocab["<eos>"]])
src_padded = torch.cat([src_ends, torch.tensor([src_vocab["<pad>"] for _ in range(src_max_len - src_ends.shape[0])])])
src = src_padded.reshape((1, -1)).to(device)

current_str = [tgt_vocab["<sos>"]]

with torch.no_grad():
    for read_token_id in range(tgt_max_len - 1):
        tgt_str = current_str + [tgt_vocab["<pad>"]] * (tgt_max_len - len(current_str) - 1)
        tgt= torch.tensor(tgt_str).to(device)
        tgt = tgt.reshape((1, -1))
        output = model(src, tgt) # (T, N, C)
        pred = output[read_token_id, 0, :]
        max_values, max_id= torch.max(pred, axis=0)
        current_str.append(max_id.item())
        if max_id == tgt_vocab["<eos>"]:
            break

list_letters = [id2token(tgt_vocab, id) for id in current_str]
print(list_letters)
print(str("".join(list_letters[1:-1])))

# Generate each token

In [ ]:

data_path = "/home/takeru/AlphaSymbol/data/prfndim/d3-a2-c3-r3-status.csv"
df = pd.read_csv(data_path)
dataset = TransformerDataset(df)

idx = 0
src, tgt, tgt_correct = dataset[idx][0], dataset[idx][1][:-1], dataset[idx][1][1:]
src, tgt, tgt_correct = src.to(device), tgt.to(device), tgt_correct.to(device)
pred_token_place= 3 #>=1

print("=== Input ===")
input_str = dataset.df["expr"].iloc[idx]    
print("input string: ", input_str)
print("pred_token_place: ", pred_token_place)

print()
print("=== Raw Data ===")
print("src: ", src)
print("tgt: ", tgt)
print("tgt_correct: ", tgt_correct)

src = src.reshape((1, -1))
pad_tensor = torch.tensor([tgt_vocab["<pad>"] for _ in range(len(tgt) - pred_token_place)]).to(device)
tgt = torch.cat((tgt[:pred_token_place], pad_tensor)).reshape((1, -1))
tgt_correct = tgt_correct[pred_token_place- 1].reshape((1, ))

print()
print("=== Processed data for inference ===")
print("src: ", src)
print("tgt: ", tgt)
print("tgt_correct: ", tgt_correct)


print()
print("=== Prediction ===")
model.eval()
output = model(src, tgt) # (Seq, N, E)
pred_token = output[pred_token_place- 1, 0, :]
print("pred: ", pred_token)
token_id =  torch.argmax(pred_token).item()
print("token_id: ",token_id)
print("token: ", id2token(tgt_vocab, token_id))


    
loss_fn = nn.CrossEntropyLoss(ignore_index=tgt_vocab["<pad>"])
pred_for_loss = pred_token.reshape((1, -1))
loss = loss_fn(pred_for_loss, tgt_correct)
print(loss)

# Generate from src, tgt

In [82]:
src = torch.tensor([[1, 3, 4, 5, 6, 4, 7, 3, 4, 5, 6, 4, 7, 3, 4, 5, 6, 4, 7, 3, 4, 5, 6, 4,
         7, 3, 4, 5, 6, 4, 7, 3, 4, 5, 6, 4, 7, 3, 4, 5, 6, 4, 7, 3, 4, 5, 6, 4,
         7, 3, 4, 5, 6, 4, 7, 3, 4, 5, 6, 4, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], device='cuda:0')

tgt = torch.tensor([[1, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],
       device='cuda:0')

output = model(src, tgt)
print(output[1, 0, :])

# Generate loss along with Training

In [ ]:
idx = 0
src, tgt, tgt_correct = dataset[idx][0], dataset[idx][1][:-1], dataset[idx][1][1:]
src = src.reshape((1, -1)).to(device)
tgt = tgt.reshape((1, -1)).to(device)
tgt_correct = tgt_correct.reshape((1, -1)).to(device)
output = model(src, tgt) # (T, N=1, C)
output = output.permute(1, 2, 0) # (N=1, C, T)
loss_fn = nn.CrossEntropyLoss(ignore_index=tgt_vocab["<pad>"])
loss = loss_fn(output, tgt_correct)
print(loss)